In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer
import pm4py
import pandas as pd

In [2]:
path = "./"

In [3]:
# These lines reads the log file
log_file_name = '2020.xes'
log_file_name_output = '2020WithBeginAndEnd.xes'
log = xes_importer.apply(path + log_file_name)

c:\workspace\.venv\lib\site-packages\pm4py\util\dt_parsing\parser.py:77: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  "ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11")


parsing log, completed traces ::   0%|          | 0/1998 [00:00<?, ?it/s]

In [4]:
df = pm4py.convert_to_dataframe(log)

c:\workspace\.venv\lib\site-packages\pm4py\utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [5]:
import pandas as pd

def add_start_end_rows(df, case_col, lifecycle_col, timestamp_col, task_col):
    # Sort the data frame by case identifier and timestamp
    df = df.sort_values(by=[case_col, timestamp_col])
    
    # Identify the starting and ending rows for each group
    starting_rows = df.groupby(case_col).apply(lambda x: x.iloc[0])
    ending_rows = df.groupby(case_col).apply(lambda x: x.iloc[-1])
    
    # Calculate the starting and ending times for each group
    starting_times = starting_rows[timestamp_col] - pd.Timedelta(minutes=1)
    ending_times = ending_rows[timestamp_col] + pd.Timedelta(minutes=1)
    
    # Identify the starting lifecycles
    lifecycles = df[lifecycle_col].unique()

    # Create the beginning and ending rows
    for cycle in lifecycles:
      beginning_rows = starting_rows.copy()
      beginning_rows[task_col] = 'BEGIN'
      beginning_rows[lifecycle_col] = cycle
      beginning_rows[timestamp_col] = starting_times
      
      ending_rows = ending_rows.copy()
      ending_rows[task_col] = 'END'
      ending_rows[lifecycle_col] = cycle
      ending_rows[timestamp_col] = ending_times
      
      # Add the beginning and ending rows to the data frame
      df = pd.concat([df, beginning_rows, ending_rows], ignore_index=True)
    
    # Sort the data frame again by case identifier and timestamp
    df = df.sort_values(by=[case_col, timestamp_col])
    
    return df



In [6]:
df = add_start_end_rows(df, 'case:concept:name', 'lifecycle:transition', 'time:timestamp', 'concept:name')

In [7]:
log = pm4py.convert_to_event_log(df)

c:\workspace\.venv\lib\site-packages\pm4py\utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [8]:
pm4py.write_xes(log, log_file_name_output)

exporting log, completed traces ::   0%|          | 0/1998 [00:00<?, ?it/s]